In [1]:
from kalah import *

# ______________________________________________________________________________
# Metodos Auxiliares

class Jogador():
    def __init__(self, nome, fun):
        self.nome = nome
        self.fun = fun
    def display(self):
        print(self.nome+" ")

class JogadorAlfaBeta(Jogador):
    def __init__(self, nome, depth,fun_eval):
        self.nome = nome
        self.fun = lambda game, state: alphabeta_cutoff_search_new(state,game,depth,eval_fn=fun_eval)

def joga11(game, jog1, jog2,verbose=False):
    ### jog1 e jog2 são jogadores com funções que dado um estado do jogo devolvem a jogada que escolheram
    ### devolve o par de jogadores, a lista de jogadas e o resultado
    estado=game.initial
    proxjog = jog1
    lista_jogadas=[]
    lance = 0
    while not game.terminal_test(estado):
        if verbose:
            print('----------   LANCE:',lance)
            game.display(estado)
        jogada = proxjog.fun(game, estado)
        if verbose:
            print('JOGADA=',jogada)
        estado=game.result(estado,jogada)
        lista_jogadas.append(jogada)
        proxjog = jog2 if proxjog == jog1 else jog1
        lance+=1
    #p jogou e ganhou
    util=game.utility(estado,0)
    if util == 1:
        resultado=jog1.nome
    elif util== -1:
        resultado = jog2.nome
    else:
        resultado='Empate'
    return ((jog1.nome,jog2.nome),lista_jogadas,resultado)


# ______________________________________________________________________________
# ______________________________________________________________________________


def func_44(estado,jogador):
    """Advanced evaluation function for the Kalah game (replace XX with your group number)."""

    # Define weights for different evaluation factors
    SCORE_WEIGHT = 1.0
    SEEDS_WEIGHT = 0.5
    PROGRESS_WEIGHT = 0.3
    PITS_WEIGHT = 0.2

    # Get the player's score and the opponent's score
    player_score = estado.state[estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.NORTH_SCORE_PIT]
    opponent_score = estado.state[estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT]

    # Calculate the number of seeds in the player's pits and the opponent's pits
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]

    # Calculate the difference in scores and seeds
    score_diff = player_score - opponent_score
    seeds_diff = sum(player_pits) - sum(opponent_pits)

    # Consider the game's progress by checking how close each player is to winning
    total_seeds = sum(player_pits) + sum(opponent_pits)
    progress = 2 * (player_score - opponent_score) / total_seeds

    # Encourage capturing opponent's pits and having fewer seeds in own pits
    player_pit_values = [min(pit, 6) for pit in player_pits]
    opponent_pit_values = [min(pit, 6) for pit in opponent_pits]
    pits_diff = sum(opponent_pit_values) - sum(player_pit_values)

    # Penalize moves that lead to the opponent capturing more seeds
    next_states = [estado.result(move) for move in estado.get_legal_moves()]
    opponent_capture_penalty = sum(sum(state[:estado.SOUTH_SCORE_PIT]) - sum(state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]) for state in next_states) * 0.05

    # Calculate the final evaluation value
    evaluation = (
        SCORE_WEIGHT * score_diff +
        SEEDS_WEIGHT * seeds_diff +
        PROGRESS_WEIGHT * progress +
        PITS_WEIGHT * pits_diff -
        opponent_capture_penalty
    )

    return evaluation



def chapiteau_algorithm(estado, jogador):
    if estado.is_game_over():
        aux = estado.result()
        return aux*100 if jogador == estado.SOUTH else aux*-100
    
    player_pits = estado.state[:estado.SOUTH_SCORE_PIT] if jogador == 0 else estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT]
    opponent_pits = estado.state[estado.SOUTH_SCORE_PIT:estado.NORTH_SCORE_PIT] if jogador == 0 else estado.state[:estado.SOUTH_SCORE_PIT]
    return sum(player_pits) - sum(opponent_pits)


In [2]:

grupo44 = JogadorAlfaBeta('Grupo 44', 5, chapiteau_algorithm)
chapiteau = JogadorAlfaBeta('Chapiteau', 5, chapiteau_algorithm)
mx = Kalah(20)
joga11(mx, grupo44, chapiteau)

(('Grupo 44', 'Chapiteau'),
 [0,
  7,
  3,
  8,
  0,
  10,
  5,
  12,
  0,
  11,
  1,
  12,
  -1,
  7,
  4,
  7,
  5,
  10,
  0,
  12,
  0,
  8,
  0,
  7,
  1,
  10,
  2,
  9,
  2,
  11,
  5,
  8,
  0,
  7,
  1,
  12,
  2,
  8],
 'Grupo 44')